In [1]:
import tensorflow as tf
import tensorflow.contrib.layers as layers
config = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, \
                        allow_soft_placement=True, device_count = {'CPU': 8})
sess = tf.InteractiveSession(config = config)

In [2]:
import numpy as np

In [3]:
# computes the KL divergence D_KL(P||Q) between the distributions of P and Q, 
# given an equal number of samples drawn from them
def KL_network(data_P, data_Q, name):
    with tf.variable_scope(name):  
        data_combined = tf.concat([data_P, data_Q], axis = 0)
        
        lay = layers.relu(data_combined, 20)
        lay = layers.relu(lay, 20)
        outputs = layers.linear(lay, 1)
        
    these_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = name)
    
    return outputs, these_vars

In [4]:
def KL_loss(KL_output, name):
    with tf.variable_scope(name):
        batch_size_dyn = tf.cast(tf.math.divide(tf.shape(KL_output)[0], 2), tf.int32)
        
        T_P = KL_output[:batch_size_dyn,:]
        T_Q = KL_output[batch_size_dyn:,:]
        
        TF_loss = -(tf.reduce_mean(T_P, axis = 0) - tf.math.log(tf.reduce_mean(tf.math.exp(T_Q), axis = 0)))
        
        TF_loss = TF_loss[0]
        
    return TF_loss

In [5]:
data_P_in = tf.placeholder(tf.float32, [None, 1], name = 'data_P_in')
data_Q_in = tf.placeholder(tf.float32, [None, 1], name = 'data_Q_in')

In [6]:
KL_output, KL_vars = KL_network(data_P_in, data_Q_in, name = 'KL')

In [7]:
KL_lossval = KL_loss(KL_output, 'KL_loss')

In [8]:
train_KL = tf.train.AdamOptimizer(learning_rate = 0.005, beta1 = 0.3, beta2 = 0.5).minimize(KL_lossval, var_list = KL_vars)

In [9]:
sess.run(tf.global_variables_initializer())

In [10]:
data_Q_train = np.random.normal(loc = 0, scale = 1, size = 50000)
data_P_train = np.random.normal(loc = 1, scale = 1, size = 50000)

In [11]:
data_Q_train = np.expand_dims(data_Q_train, axis = 1)
data_P_train = np.expand_dims(data_P_train, axis = 1)

In [15]:
for epoch in range(100):
    sess.run(train_KL, feed_dict = {data_P_in: data_P_train, data_Q_in: data_Q_train})
    loss = -sess.run(KL_lossval, feed_dict = {data_P_in: data_P_train, data_Q_in: data_Q_train})
    print("epoch {}: loss = {}".format(epoch, loss))

epoch 0: loss = 0.5036814212799072
epoch 1: loss = 0.5039948225021362
epoch 2: loss = 0.5043094158172607
epoch 3: loss = 0.5043565034866333
epoch 4: loss = 0.5043433904647827
epoch 5: loss = 0.5039451122283936
epoch 6: loss = 0.503258228302002
epoch 7: loss = 0.5041403770446777
epoch 8: loss = 0.5043749809265137
epoch 9: loss = 0.5043420791625977
epoch 10: loss = 0.5040558576583862
epoch 11: loss = 0.5034643411636353
epoch 12: loss = 0.5040665864944458
epoch 13: loss = 0.5043784379959106
epoch 14: loss = 0.5044025182723999
epoch 15: loss = 0.5043841600418091
epoch 16: loss = 0.5039399862289429
epoch 17: loss = 0.5031805038452148
epoch 18: loss = 0.5042284727096558
epoch 19: loss = 0.5044219493865967
epoch 20: loss = 0.5043220520019531
epoch 21: loss = 0.5039271116256714
epoch 22: loss = 0.5036430358886719
epoch 23: loss = 0.5042078495025635
epoch 24: loss = 0.5044084787368774
epoch 25: loss = 0.5044209957122803
epoch 26: loss = 0.5043830871582031
epoch 27: loss = 0.5037965774536133
epo